In [1]:
import cv2
import numpy as np
import pandas as pd
import math
import os
import tensorflow as tf

# טוען את המודל movenet
interpreter = tf.lite.Interpreter(model_path="thunder3.tflite")
interpreter.allocate_tensors()

def calculate_angle(p1, p2, p3):
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)
    angle = np.arccos(
        np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
    )
    return np.degrees(angle)

def normalize_data(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# פונקציה לחישוב ציון תנועה על פי הנוסחה שניתנה
def calculate_movement_score(velocity_avg, angle_score, confidence_score):
    return 0.4 * velocity_avg + 0.3 * angle_score + 0.3 * confidence_score

# פונקציה לחישוב ציוני תנועה עבור חלון זמן
def calculate_scores_for_window(window_data):
    # חישוב ממוצעים וציונים עבור כל חלקי גוף
    
    # כתף שמאל - רק מהירות וביטחון
    left_shoulder_velocity_avg = np.mean(window_data["left_shoulder_velocities"]) if window_data["left_shoulder_velocities"] else 0
    left_shoulder_confidence_avg = np.mean(window_data["left_shoulder_confidences"]) if window_data["left_shoulder_confidences"] else 0
    
    # כתף ימין - רק מהירות וביטחון
    right_shoulder_velocity_avg = np.mean(window_data["right_shoulder_velocities"]) if window_data["right_shoulder_velocities"] else 0
    right_shoulder_confidence_avg = np.mean(window_data["right_shoulder_confidences"]) if window_data["right_shoulder_confidences"] else 0
    
    # מרפק שמאל - מהירות, זווית וביטחון
    left_elbow_velocity_avg = np.mean(window_data["left_elbow_velocities"]) if window_data["left_elbow_velocities"] else 0
    left_elbow_angle_avg = np.mean(window_data["left_elbow_angles"]) if window_data["left_elbow_angles"] else 0
    left_elbow_confidence_avg = np.mean(window_data["left_elbow_confidences"]) if window_data["left_elbow_confidences"] else 0
    
    # מרפק ימין - מהירות, זווית וביטחון
    right_elbow_velocity_avg = np.mean(window_data["right_elbow_velocities"]) if window_data["right_elbow_velocities"] else 0
    right_elbow_angle_avg = np.mean(window_data["right_elbow_angles"]) if window_data["right_elbow_angles"] else 0
    right_elbow_confidence_avg = np.mean(window_data["right_elbow_confidences"]) if window_data["right_elbow_confidences"] else 0
    
    # ירך שמאל - רק מהירות וביטחון
    left_hip_velocity_avg = np.mean(window_data["left_hip_velocities"]) if window_data["left_hip_velocities"] else 0
    left_hip_confidence_avg = np.mean(window_data["left_hip_confidences"]) if window_data["left_hip_confidences"] else 0
    
    # ירך ימין - רק מהירות וביטחון
    right_hip_velocity_avg = np.mean(window_data["right_hip_velocities"]) if window_data["right_hip_velocities"] else 0
    right_hip_confidence_avg = np.mean(window_data["right_hip_confidences"]) if window_data["right_hip_confidences"] else 0
    
    # ברך שמאל - מהירות, זווית וביטחון
    left_knee_velocity_avg = np.mean(window_data["left_knee_velocities"]) if window_data["left_knee_velocities"] else 0
    left_knee_angle_avg = np.mean(window_data["left_knee_angles"]) if window_data["left_knee_angles"] else 0
    left_knee_confidence_avg = np.mean(window_data["left_knee_confidences"]) if window_data["left_knee_confidences"] else 0
    
    # ברך ימין - מהירות, זווית וביטחון
    right_knee_velocity_avg = np.mean(window_data["right_knee_velocities"]) if window_data["right_knee_velocities"] else 0
    right_knee_angle_avg = np.mean(window_data["right_knee_angles"]) if window_data["right_knee_angles"] else 0
    right_knee_confidence_avg = np.mean(window_data["right_knee_confidences"]) if window_data["right_knee_confidences"] else 0
    
    # חישוב ציוני תנועה עבור ברכיים ומרפקים (לקביעת דומיננטיות)
    left_elbow_score = calculate_movement_score(left_elbow_velocity_avg, left_elbow_angle_avg, left_elbow_confidence_avg)
    right_elbow_score = calculate_movement_score(right_elbow_velocity_avg, right_elbow_angle_avg, right_elbow_confidence_avg)
    left_knee_score = calculate_movement_score(left_knee_velocity_avg, left_knee_angle_avg, left_knee_confidence_avg)
    right_knee_score = calculate_movement_score(right_knee_velocity_avg, right_knee_angle_avg, right_knee_confidence_avg)
    
    # עבור איברים ללא זווית (כתפיים וירכיים), נשתמש בערך זווית 0 לצורך הנוסחה
    left_shoulder_score = calculate_movement_score(left_shoulder_velocity_avg, 0, left_shoulder_confidence_avg)
    right_shoulder_score = calculate_movement_score(right_shoulder_velocity_avg, 0, right_shoulder_confidence_avg)
    left_hip_score = calculate_movement_score(left_hip_velocity_avg, 0, left_hip_confidence_avg)
    right_hip_score = calculate_movement_score(right_hip_velocity_avg, 0, right_hip_confidence_avg)
    
    # חישוב ציון כללי כממוצע של כל הציונים מכל האיברים
    overall_score = (
        left_elbow_score + right_elbow_score + 
        left_knee_score + right_knee_score + 
        left_shoulder_score + right_shoulder_score + 
        left_hip_score + right_hip_score
    ) / 8
    
    # קביעת איבר דומיננטי
    dominant_knee = "left" if left_knee_score > right_knee_score else "right"
    dominant_elbow = "left" if left_elbow_score > right_elbow_score else "right"
    
    return {
        "overall_movement_score": overall_score,
        "left_knee_score": left_knee_score,
        "right_knee_score": right_knee_score,
        "dominant_knee": dominant_knee,
        "left_elbow_score": left_elbow_score,
        "right_elbow_score": right_elbow_score,
        "dominant_elbow": dominant_elbow,
        "window_start_frame": window_data["start_frame"],
        "window_end_frame": window_data["end_frame"]
    }

video_folder = "final_videos"
video_files = sorted([f for f in os.listdir(video_folder) if f.endswith(".mp4")])

dataset = []

# רק הנקודות שנבחרו
selected_keypoints = [5, 6, 7, 8, 11, 12, 13, 14]

MIN_X = -1
MAX_X = 1
MIN_Y = -1
MAX_Y = 1
MIN_VELOCITY = -10
MAX_VELOCITY = 10
MIN_ACCELERATION = -10
MAX_ACCELERATION = 10

# גודל חלון הזמן בפריימים
WINDOW_SIZE = 60

for video_index, video_file in enumerate(video_files):
    video_id = video_index + 1
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    prev_keypoints = None
    prev_velocity = None
    prev_time = None
    frame_index = 0
    
    # מילון לאחסון נתונים לכל חלון זמן
    time_windows_data = {}
    current_window_index = 0
    
    # איתחול חלון הזמן הראשון
    time_windows_data[current_window_index] = {
        "start_frame": 0,
        "end_frame": WINDOW_SIZE - 1,
        "video_id": video_id,
        # נתוני כתף שמאל (keypoint_5)
        "left_shoulder_velocities": [],
        "left_shoulder_confidences": [],
        # נתוני כתף ימין (keypoint_6)
        "right_shoulder_velocities": [],
        "right_shoulder_confidences": [],
        # נתוני מרפק שמאל (keypoint_7)
        "left_elbow_velocities": [],
        "left_elbow_angles": [],
        "left_elbow_confidences": [],
        # נתוני מרפק ימין (keypoint_8)
        "right_elbow_velocities": [],
        "right_elbow_angles": [],
        "right_elbow_confidences": [],
        # נתוני ירך שמאל (keypoint_11)
        "left_hip_velocities": [],
        "left_hip_confidences": [],
        # נתוני ירך ימין (keypoint_12)
        "right_hip_velocities": [],
        "right_hip_confidences": [],
        # נתוני ברך שמאל (keypoint_13)
        "left_knee_velocities": [],
        "left_knee_angles": [],
        "left_knee_confidences": [],
        # נתוני ברך ימין (keypoint_14)
        "right_knee_velocities": [],
        "right_knee_angles": [],
        "right_knee_confidences": []
    }

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # בדיקה אם צריך לפתוח חלון זמן חדש
        if frame_index >= (current_window_index + 1) * WINDOW_SIZE:
            current_window_index += 1
            time_windows_data[current_window_index] = {
                "start_frame": current_window_index * WINDOW_SIZE,
                "end_frame": (current_window_index + 1) * WINDOW_SIZE - 1,
                "video_id": video_id,
                # איתחול מחדש של כל המערכים
                "left_shoulder_velocities": [],
                "left_shoulder_confidences": [],
                "right_shoulder_velocities": [],
                "right_shoulder_confidences": [],
                "left_elbow_velocities": [],
                "left_elbow_angles": [],
                "left_elbow_confidences": [],
                "right_elbow_velocities": [],
                "right_elbow_angles": [],
                "right_elbow_confidences": [],
                "left_hip_velocities": [],
                "left_hip_confidences": [],
                "right_hip_velocities": [],
                "right_hip_confidences": [],
                "left_knee_velocities": [],
                "left_knee_angles": [],
                "left_knee_confidences": [],
                "right_knee_velocities": [],
                "right_knee_angles": [],
                "right_knee_confidences": []
            }

        input_tensor = cv2.resize(frame, (256, 256))
        input_tensor = np.expand_dims(input_tensor.astype(np.float32), axis=0)

        interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_tensor)
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

        keypoints = keypoints_with_scores[0, 0, :, :2]
        scores = keypoints_with_scores[0, 0, :, 2]

        height, width, _ = frame.shape
        keypoints = keypoints * np.array([width, height])

        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        velocity = None
        acceleration = None

        if prev_keypoints is not None and prev_time is not None:
            time_diff = current_time - prev_time
            if time_diff > 0:
                velocity = (keypoints - prev_keypoints) / time_diff
                if prev_velocity is not None:
                    acceleration = (velocity - prev_velocity) / time_diff

        angles = {
            13: ("left_knee_angle", calculate_angle(keypoints[11], keypoints[13], keypoints[15])),
            14: ("right_knee_angle", calculate_angle(keypoints[12], keypoints[14], keypoints[16])),
            7: ("left_elbow_angle", calculate_angle(keypoints[5], keypoints[7], keypoints[9])),
            8: ("right_elbow_angle", calculate_angle(keypoints[6], keypoints[8], keypoints[10]))
        }

        data_entry = {
            "video_id": video_id,
            "frame": frame_index,
            "window_index": current_window_index
        }

        for i in selected_keypoints:
            point = keypoints[i]
            score = scores[i]
            norm_x = normalize_data(point[0], MIN_X, MAX_X)
            norm_y = normalize_data(point[1], MIN_Y, MAX_Y)
            data_entry[f"keypoint_{i}_x"] = norm_x
            data_entry[f"keypoint_{i}_y"] = norm_y
            data_entry[f"keypoint_{i}_confidence"] = score

            # שמירת הוודאות עבור חישוב ציון התנועה בחלון הזמן הנוכחי
            if i == 5:  # כתף שמאל
                time_windows_data[current_window_index]["left_shoulder_confidences"].append(score)
            elif i == 6:  # כתף ימין
                time_windows_data[current_window_index]["right_shoulder_confidences"].append(score)
            elif i == 11:  # ירך שמאל
                time_windows_data[current_window_index]["left_hip_confidences"].append(score)
            elif i == 12:  # ירך ימין
                time_windows_data[current_window_index]["right_hip_confidences"].append(score)

            if i in angles:
                angle_name, angle_value = angles[i]
                data_entry[angle_name] = angle_value
                
                # שמור את הזווית והוודאות עבור חישוב ציון התנועה בחלון הזמן הנוכחי
                if i == 7:  # מרפק שמאל
                    time_windows_data[current_window_index]["left_elbow_angles"].append(angle_value)
                    time_windows_data[current_window_index]["left_elbow_confidences"].append(score)
                elif i == 8:  # מרפק ימין
                    time_windows_data[current_window_index]["right_elbow_angles"].append(angle_value)
                    time_windows_data[current_window_index]["right_elbow_confidences"].append(score)
                elif i == 13:  # ברך שמאל
                    time_windows_data[current_window_index]["left_knee_angles"].append(angle_value)
                    time_windows_data[current_window_index]["left_knee_confidences"].append(score)
                elif i == 14:  # ברך ימין
                    time_windows_data[current_window_index]["right_knee_angles"].append(angle_value)
                    time_windows_data[current_window_index]["right_knee_confidences"].append(score)

            if velocity is not None:
                vel_magnitude = np.linalg.norm(velocity[i])
                norm_velocity_x = normalize_data(velocity[i][0], MIN_VELOCITY, MAX_VELOCITY)
                norm_velocity_y = normalize_data(velocity[i][1], MIN_VELOCITY, MAX_VELOCITY)
                data_entry[f"keypoint_{i}_velocity_x"] = norm_velocity_x
                data_entry[f"keypoint_{i}_velocity_y"] = norm_velocity_y
                
                # שמור את המהירות עבור חישוב ציון התנועה בחלון הזמן הנוכחי
                if i == 5:  # כתף שמאל
                    time_windows_data[current_window_index]["left_shoulder_velocities"].append(vel_magnitude)
                elif i == 6:  # כתף ימין
                    time_windows_data[current_window_index]["right_shoulder_velocities"].append(vel_magnitude)
                elif i == 7:  # מרפק שמאל
                    time_windows_data[current_window_index]["left_elbow_velocities"].append(vel_magnitude)
                elif i == 8:  # מרפק ימין
                    time_windows_data[current_window_index]["right_elbow_velocities"].append(vel_magnitude)
                elif i == 11:  # ירך שמאל
                    time_windows_data[current_window_index]["left_hip_velocities"].append(vel_magnitude)
                elif i == 12:  # ירך ימין
                    time_windows_data[current_window_index]["right_hip_velocities"].append(vel_magnitude)
                elif i == 13:  # ברך שמאל
                    time_windows_data[current_window_index]["left_knee_velocities"].append(vel_magnitude)
                elif i == 14:  # ברך ימין
                    time_windows_data[current_window_index]["right_knee_velocities"].append(vel_magnitude)

            if acceleration is not None:
                norm_acceleration_x = normalize_data(acceleration[i][0], MIN_ACCELERATION, MAX_ACCELERATION)
                norm_acceleration_y = normalize_data(acceleration[i][1], MIN_ACCELERATION, MAX_ACCELERATION)
                data_entry[f"keypoint_{i}_acceleration_x"] = norm_acceleration_x
                data_entry[f"keypoint_{i}_acceleration_y"] = norm_acceleration_y

        dataset.append(data_entry)
        prev_keypoints = keypoints.copy()
        prev_velocity = velocity.copy() if velocity is not None else None
        prev_time = current_time
        frame_index += 1

    cap.release()

    # חישוב ציוני תנועה לכל חלון זמן
    window_scores = {}
    for window_index, window_data in time_windows_data.items():
        # אם יש פחות מ-5 פריימים בחלון, דלג על החישוב (למנוע חישוב על חלונות חלקיים מדי)
        if len(window_data["left_knee_confidences"]) < 5:
            continue
        window_scores[window_index] = calculate_scores_for_window(window_data)

# הוספת הציונים לכל שורה בדאטאסט
for data_entry in dataset:
    window_index = data_entry["window_index"]
    if window_index in window_scores:
        scores = window_scores[window_index]
        data_entry["overall_movement_score"] = scores["overall_movement_score"]
        data_entry["left_knee_score"] = scores["left_knee_score"]
        data_entry["right_knee_score"] = scores["right_knee_score"]
        data_entry["dominant_knee"] = scores["dominant_knee"]
        data_entry["left_elbow_score"] = scores["left_elbow_score"]
        data_entry["right_elbow_score"] = scores["right_elbow_score"]
        data_entry["dominant_elbow"] = scores["dominant_elbow"]
    else:
        # אם אין ציונים לחלון זה (כנראה חלון חלקי בסוף הסרטון)
        data_entry["overall_movement_score"] = np.nan
        data_entry["left_knee_score"] = np.nan
        data_entry["right_knee_score"] = np.nan
        data_entry["dominant_knee"] = ""
        data_entry["left_elbow_score"] = np.nan
        data_entry["right_elbow_score"] = np.nan
        data_entry["dominant_elbow"] = ""

# יצירת הדאטאפריים וכתיבה לקובץ CSV
df = pd.DataFrame(dataset)
df.to_csv("movenet_motion_dataset_with_window_scores.csv", index=False)
print("✅ סיום עיבוד הסרטונים, הנתונים נשמרו בקובץ movenet_motion_dataset_with_window_scores.csv")

C:\Users\Ronen\AppData\Roaming\Python\Python312\site-packages\tensorflow\lite\python\interpreter.py:451: UserWarning:     Warning: Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ סיום עיבוד הסרטונים, הנתונים נשמרו בקובץ movenet_motion_dataset_with_window_scores.csv
